In [36]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [2]:
config = read_config('.env/local_conn.json')
sql_agent = MySQLAgent(config['local_mysql'])

In [3]:
query = """
show tables
"""

sql_agent.read_table(query=query)

,tables_in_generaldb
0,balancesheet
1,profitandlose


### Helper functions

In [4]:
def get_acct_data_and_rename(df, row_name, new_name):

    result = df[df['acct_name'] == row_name][['year_season', 'this_year_amt']].reset_index(drop=True).rename(columns={'this_year_amt':new_name})
    result[new_name] = result[new_name].str.replace(',', '').astype(int)

    return result

In [5]:
query = """
select *
from balancesheet
where stock = '2330' and season = 4
"""

df_balance = sql_agent.read_table(query=query)

In [6]:
df_balance.acct_name.unique()

array(['流動資產', '現金及約當現金', '透過損益按公允價值衡量之金融資產－流動',
       '透過其他綜合損益按公允價值衡量之金融資產－流動', '按攤銷後成本衡量之金融資產－流動', '避險之金融資產－流動',
       '應收帳款淨額', '應收帳款－關係人淨額', '其他應收款－關係人淨額', '存貨', '其他流動資產', '流動資產合計',
       '非流動資產', '透過其他綜合損益按公允價值衡量之金融資產－非流動', '按攤銷後成本衡量之金融資產－非流動',
       '採用權益法之投資', '不動產、廠房及設備', '使用權資產', '無形資產', '遞延所得稅資產', '其他非流動資產',
       '非流動資產合計', '資產總額', '流動負債', '短期借款', '透過損益按公允價值衡量之金融負債－流動',
       '避險之金融負債－流動', '應付帳款', '應付帳款－關係人', '其他應付款', '本期所得稅負債', '其他流動負債',
       '流動負債合計', '非流動負債', '應付公司債', '遞延所得稅負債', '租賃負債－非流動', '其他非流動負債',
       '非流動負債合計', '負債總額', '歸屬於母公司業主之權益', '股本', '普通股股本', '股本合計', '資本公積',
       '資本公積－發行溢價', '資本公積-認列對子公司所有權權益變動數', '資本公積－受贈資產',
       '資本公積－採用權益法認列關聯企業及合資股權淨值之變動數', '資本公積－合併溢額', '資本公積合計', '保留盈餘',
       '法定盈餘公積', '特別盈餘公積', '未分配盈餘（或待彌補虧損）', '保留盈餘合計', '其他權益', '其他權益合計',
       '歸屬於母公司業主之權益合計', '非控制權益', '權益總額', '負債及權益總計',
       '預收股款（權益項下）之約當發行股數（單位：股）', '母公司暨子公司所持有之母公司庫藏股股數（單位：股）', '長期借款',
       '資本公積－實際取得或處分子公司股權價格與帳面價值差額', '資本公積－限制員工權利股票', '庫藏股票',
       '透過損益按公允價

In [7]:
query = """
select *
from profitandlose
where stock = '2330' and season = 4
"""

df_profitloss = sql_agent.read_table(query=query)

In [8]:
df_profitloss

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
0,營業收入合計,"1,069,985,448",100.00,"1,031,473,557",100.00,ProfitAndLose,民國108年第4季,108,4,2024-08-09,108-4,2330
1,營業成本合計,"577,286,947",53.95,"533,487,516",51.72,ProfitAndLose,民國108年第4季,108,4,2024-08-09,108-4,2330
2,營業毛利（毛損）,"492,698,501",46.05,"497,986,041",48.28,ProfitAndLose,民國108年第4季,108,4,2024-08-09,108-4,2330
3,未實現銷貨（損）益,"-3,395",0.00,"111,788",0.01,ProfitAndLose,民國108年第4季,108,4,2024-08-09,108-4,2330
4,營業毛利（毛損）淨額,"492,701,896",46.05,"497,874,253",48.27,ProfitAndLose,民國108年第4季,108,4,2024-08-09,108-4,2330
...,...,...,...,...,...,...,...,...,...,...,...,...
228,非控制權益（綜合損益）,"-1,555,669",-0.07,"205,790",0.01,ProfitAndLose,民國112年第4季,112,4,2024-08-09,112-4,2330
229,基本每股盈餘,,,,,ProfitAndLose,民國112年第4季,112,4,2024-08-09,112-4,2330
230,基本每股盈餘,32.34,,39.20,,ProfitAndLose,民國112年第4季,112,4,2024-08-09,112-4,2330
231,稀釋每股盈餘,,,,,ProfitAndLose,民國112年第4季,112,4,2024-08-09,112-4,2330


# Financial Structure

### Debut Burden Ratio(DBR) (資產負債比)

In [9]:
# 資產總額
df_total_assests = get_acct_data_and_rename(df_balance, '資產總額', 'total_assests')
df_total_assests['total_assests_varation'] = df_total_assests['total_assests'].pct_change()*100
df_total_assests

,year_season,total_assests,total_assests_varation
0,108-4,2264805032,NaN
1,109-4,2760711405,21.896206
2,110-4,3725503455,34.947226
3,111-4,4964778878,33.264643
4,112-4,5532371215,11.432379


In [10]:
# 負債總額
df_total_liability = get_acct_data_and_rename(df_balance, '負債總額', 'total_liabilities')
df_total_liability['total_liabilities_varation'] = df_total_liability['total_liabilities'].pct_change()*100
df_total_liability

,year_season,total_liabilities,total_liabilities_varation
0,108-4,642709606,NaN
1,109-4,910089406,41.601961
2,110-4,1554770250,70.837089
3,111-4,2004290011,28.912295
4,112-4,2049108368,2.236121


In [11]:
# DBR: Debut Burden Ratio
df_total_assests_liability = pd.merge(df_total_assests, df_total_liability, how='inner', on='year_season')
df_total_assests_liability['DBR'] = (round(df_total_assests_liability['total_liabilities'].astype(int) / df_total_assests_liability['total_assests'].astype(int) * 100, 2)).astype(str) + '%'
df_total_assests_liability

,year_season,total_assests,total_assests_varation,total_liabilities,total_liabilities_varation,DBR
0,108-4,2264805032,NaN,642709606,NaN,28.38%
1,109-4,2760711405,21.896206,910089406,41.601961,32.97%
2,110-4,3725503455,34.947226,1554770250,70.837089,41.73%
3,111-4,4964778878,33.264643,2004290011,28.912295,40.37%
4,112-4,5532371215,11.432379,2049108368,2.236121,37.04%


- 108年~110年，債務比例增加，可能是要擴廠或增加設備等等

### Long term funds to fixed assets ratio(長期資金佔固定資產比率)

(股東權益淨額 + 長期負債) / 固定資產淨額

在評估企業財務穩健性時，長期資金佔固定資產比率是一個重要的指標。長期資金，如股本和長期負債，被視為相對穩定的資金來源，而固定資產是指長期持有、無法迅速變現的資產，如廠房和設備。長期資金佔固定資產比率衡量長期資金對固定資產的覆蓋程度，反映企業償還固定資產投資的能力。

一般而言，較高的長期資金佔固定資產比率表示企業的財務結構較為穩健，因為它意味著企業有更多資金可供償還固定資產投資。然而，過高的比率也並非好事。當長期資金過多時，企業可能會無法有效利用資金，導致資金利用率低下，進而影響盈利能力。因此，在評估企業的財務穩健性時，投資者需要綜合考量長期資金佔固定資產比率和其他財務指標，例如負債對股東權益比率和現金流量狀況，才能做出全面判斷。

In [12]:
# 固定資產: 不動產、廠房及設備
df_fixed_assets = get_acct_data_and_rename(df_balance, '不動產、廠房及設備', 'fixed_assests')
df_fixed_assets

,year_season,fixed_assests
0,108-4,1352377405
1,109-4,1555589120
2,110-4,1975118704
3,111-4,2693836970
4,112-4,3064474984


In [15]:
# 長期借款
df_long_term_liabilities = get_acct_data_and_rename(df_balance, '長期借款', 'long-term_liabilities')
df_long_term_liabilities

,year_season,long-term_liabilities
0,109-4,1967611
1,110-4,3309131
2,111-4,4760047
3,112-4,4382965


In [16]:
df_balance[df_balance['acct_name'] == '長期借款']

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
99,長期借款,"1,967,611",0.07,0,0.00,BalanceSheet,民國109年第4季,109,4,2024-08-09,109-4,2330
164,長期借款,"3,309,131",0.09,"1,967,611",0.07,BalanceSheet,民國110年第4季,110,4,2024-08-09,110-4,2330
230,長期借款,"4,760,047",0.10,"3,309,131",0.09,BalanceSheet,民國111年第4季,111,4,2024-08-09,111-4,2330
298,長期借款,"4,382,965",0.08,"4,760,047",0.10,BalanceSheet,民國112年第4季,112,4,2024-08-09,112-4,2330


In [17]:
# 股東權益＝資產－負債
df_shareholder_equity = df_total_assests_liability[['year_season', 'total_assests', 'total_liabilities']].copy()
df_shareholder_equity['shareholder_equity'] = df_shareholder_equity['total_assests'] - df_shareholder_equity['total_liabilities']
df_shareholder_equity

,year_season,total_assests,total_liabilities,shareholder_equity
0,108-4,2264805032,642709606,1622095426
1,109-4,2760711405,910089406,1850621999
2,110-4,3725503455,1554770250,2170733205
3,111-4,4964778878,2004290011,2960488867
4,112-4,5532371215,2049108368,3483262847


In [18]:
parent_table = df_balance[['year_season']].drop_duplicates()
df_temp1 = pd.merge(df_fixed_assets, df_long_term_liabilities, how='left', on='year_season')
df_temp2 = pd.merge(df_temp1, df_shareholder_equity, how='left', on='year_season')
# in here, we should check the last amt in the original table. 0 here is the result after checking
# create a check function laterly
df_longterm_funds_to_fixed_assets_retio = pd.merge(parent_table, df_temp2, how='left', on='year_season').fillna(0)
df_longterm_funds_to_fixed_assets_retio['longterm_funds_to_fixed_assets_retio'] = round((df_longterm_funds_to_fixed_assets_retio['long-term_liabilities'].astype(int) + df_longterm_funds_to_fixed_assets_retio['shareholder_equity'])/ df_longterm_funds_to_fixed_assets_retio['fixed_assests'].astype(int), 2)
df_longterm_funds_to_fixed_assets_retio

,year_season,fixed_assests,long-term_liabilities,total_assests,total_liabilities,shareholder_equity,longterm_funds_to_fixed_assets_retio
0,108-4,1352377405,0.0,2264805032,642709606,1622095426,1.20
1,109-4,1555589120,1967611.0,2760711405,910089406,1850621999,1.19
2,110-4,1975118704,3309131.0,3725503455,1554770250,2170733205,1.10
3,111-4,2693836970,4760047.0,4964778878,2004290011,2960488867,1.10
4,112-4,3064474984,4382965.0,5532371215,2049108368,3483262847,1.14


- The longterm_funds_to_fixed_assets_ratio for TSMC has fluctuated slightly between 1.10 and 1.20 over the past five years, indicating a balanced and consistent financial strategy.
- The slight decrease in the ratio in 2021 and 2022 reflects strategic investments in fixed assets, possibly for expansion or upgrading capabilities.
- The increase in the ratio in 2023 suggests a return to a slightly more conservative approach, increasing long-term funds relative to fixed assets, which enhances financial stability.
- The consistent ratios indicate that TSMC effectively manages its long-term funds to ensure sufficient coverage for its fixed assets, supporting sustained growth and operational resilience.

# Solvency (償債能力)

### Current ratio (流動比率)

Current ratio = current assets / current liabilities

流動比率 = 流動資產 / 流動負債

In [19]:
df_current_assets = get_acct_data_and_rename(df_balance, '流動資產合計', 'current_assets')
df_current_assets

,year_season,current_assets
0,108-4,822613914
1,109-4,1092185308
2,110-4,1607072907
3,111-4,2052896744
4,112-4,2194032910


In [20]:
df_current_liabilities = get_acct_data_and_rename(df_balance, '流動負債合計', 'current_liabilities')
df_current_liabilities

,year_season,current_liabilities
0,108-4,590735701
1,109-4,617151048
2,110-4,739503358
3,111-4,944226817
4,112-4,913583316


In [21]:
df_assets_liabilities = pd.merge(df_current_assets, df_current_liabilities, how='inner', on='year_season')
df_assets_liabilities['current_ratio'] = (round(df_assets_liabilities['current_assets'] / df_assets_liabilities['current_liabilities'] *100, 2)).astype(str) + '%'
df_assets_liabilities

,year_season,current_assets,current_liabilities,current_ratio
0,108-4,822613914,590735701,139.25%
1,109-4,1092185308,617151048,176.97%
2,110-4,1607072907,739503358,217.32%
3,111-4,2052896744,944226817,217.42%
4,112-4,2194032910,913583316,240.16%


- Current ratio is bigger than 100% which is healthy.
- The meaning of healthy is that the company has the ability to pay the liabilities.

流動比率大於100%為佳， 若小於100%須瞭解流動資產的變現性， 還有短期金融負債(短期借款、應附商業本票和一年內到期長期負債)佔流動負債的比例。 如果流動資產變現性很高，短期金融負債佔流動負債的比例低， 則低流動比率可接受；反之則償債能力不佳。 短期償債壓力越大，投資應特別注意。

In [22]:
# def for calculating short term liabilities

### Quick Ratio(速動比率)

速動比率 =（流動資產-存貨）/流動負債

In [23]:
# 存貨
df_inventories = get_acct_data_and_rename(df_balance, '存貨', 'inventories')
df_inventories

,year_season,inventories
0,108-4,82981196
1,109-4,137353407
2,110-4,193102321
3,111-4,221149148
4,112-4,250997088


In [24]:
df_assets_liabilities = pd.merge(df_current_assets, df_current_liabilities, how='inner', on='year_season')
df_assets_liabilities_inventories = pd.merge(df_assets_liabilities, df_inventories, how='inner', on='year_season')
df_assets_liabilities_inventories['quick_ratio'] = round((df_assets_liabilities_inventories['current_assets'] - df_assets_liabilities_inventories['inventories'])/df_assets_liabilities_inventories['current_liabilities']*100, 2).astype(str) + '%'
df_assets_liabilities_inventories

,year_season,current_assets,current_liabilities,inventories,quick_ratio
0,108-4,822613914,590735701,82981196,125.21%
1,109-4,1092185308,617151048,137353407,154.72%
2,110-4,1607072907,739503358,193102321,191.21%
3,111-4,2052896744,944226817,221149148,193.99%
4,112-4,2194032910,913583316,250997088,212.68%


# Operating ability(經營能力)

### Receivables Turnover Ratio(應收款項週轉率)

- 銷貨淨額: Revenue 
- 各期平均應收款項餘額: Average Accounts Receivable
- DSO = (Average Accounts Receivable) / (Revenue) * 365

- Receivables Turnover Ratio = (Revenue) / Average Accounts Receivable
- 應收款項週轉率 = 銷貨淨額 / 各期平均應收款項餘額

In [25]:
# 應收帳款淨額
df_receivables = get_acct_data_and_rename(df_balance, '應收帳款淨額', 'receivables')
df_receivables

,year_season,receivables
0,108-4,138908589
1,109-4,145480272
2,110-4,197586109
3,111-4,229755887
4,112-4,201313914


In [28]:
# 銷貨淨額
df_net_revenue = get_acct_data_and_rename(df_profitloss, '營業毛利（毛損）淨額', 'net_revenue')
df_net_revenue

,year_season,net_revenue
0,108-4,492701896
1,109-4,711130120
2,110-4,819537266
3,111-4,1348354806
4,112-4,1175110628


In [29]:
df_receivable_turnover = pd.merge(df_receivables, df_net_revenue, how='inner', on='year_season')
df_receivable_turnover['receivable_turnover_ratio'] = (df_receivable_turnover['receivables'] / df_receivable_turnover['net_revenue'])
df_receivable_turnover

,year_season,receivables,net_revenue,receivable_turnover_ratio
0,108-4,138908589,492701896,0.281932
1,109-4,145480272,711130120,0.204576
2,110-4,197586109,819537266,0.241095
3,111-4,229755887,1348354806,0.170397
4,112-4,201313914,1175110628,0.171315


### Inventory Turnover(存貨周轉率)

公司在某一段時間的營業成本與平均存貨餘額的比例，可以反應存貨的周轉速度，藉此看出存貨流動性、存貨資金占用量是否合理。

存貨周轉率 = 營業成本 / 平均存貨 = 營業成本 / ((期初存貨 + 期末存貨) / 2)

Inventroy Turnover = 

In [32]:
df_operation_cost = get_acct_data_and_rename(df_profitloss, '營業成本合計', 'operation_cost')
df_operation_cost

,year_season,operation_cost
0,108-4,577286947
1,109-4,628108309
2,110-4,767877771
3,111-4,915536486
4,112-4,986625213


In [33]:
df_inventories = get_acct_data_and_rename(df_balance, '存貨', 'inventories')
df_inventories

,year_season,inventories
0,108-4,82981196
1,109-4,137353407
2,110-4,193102321
3,111-4,221149148
4,112-4,250997088


In [37]:
df_inventories['avg_inventories'] = (df_inventories['inventories'] + df_inventories['inventories'].shift(1)) / 2
df_inventories['avg_inventories'] = np.where(df_inventories['avg_inventories'].isnull(), df_inventories['inventories'], df_inventories['avg_inventories'])
df_inventories

,year_season,inventories,avg_inventories
0,108-4,82981196,82981196.0
1,109-4,137353407,110167301.5
2,110-4,193102321,165227864.0
3,111-4,221149148,207125734.5
4,112-4,250997088,236073118.0


In [44]:
df_inventories_turnover = pd.merge(df_inventories, df_operation_cost, how='inner', on='year_season')
df_inventories_turnover['inventories_turnover'] = round(df_inventories_turnover['operation_cost'] / df_inventories_turnover['avg_inventories'], 2)
# Days sales of invetory
df_inventories_turnover['DSI'] = round(365 / df_inventories_turnover['inventories_turnover'], 2)
df_inventories_turnover

,year_season,inventories,avg_inventories,operation_cost,inventories_turnover,DSI
0,108-4,82981196,82981196.0,577286947,6.96,52.44
1,109-4,137353407,110167301.5,628108309,5.70,64.04
2,110-4,193102321,165227864.0,767877771,4.65,78.49
3,111-4,221149148,207125734.5,915536486,4.42,82.58
4,112-4,250997088,236073118.0,986625213,4.18,87.32
